# About this code

- Code author: Chenguang Wang     
- Email: c.wang-8@tudelft.nl; samwangchenguang@gmail.com
- Affiliation: Delft University of Technology
- Project name: Generating multivariate load states using a (conditional) variational autoencoder
- Motivation: This is a project for PSCC2022 – Power Systems Computation Conference: [Homepage of the conference](https://pscc2022.pt/)
- Aim of this code: Generating 32-dimensional load states using a conditional variational autoencoder with fixed output noise parameters (Manually set σ')
- A preprint is available, and you can check this paper for more details  [Link of the paper](https://arxiv.org/abs/2110.11435)
    - Paper authors: Chenguang Wang, Ensieh Sharifnia, Zhi Gao, Simon H. Tindemans, Peter Palensky
    - Accepted for publication at PSCC 2022 and a special issue of EPSR
    - If you use (parts of) this code, please cite the preprint or published paper

# Preparation (Library)

In [ ]:
# ==================== Preparation (library) =======================
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Preparation (Data)
- Input: Load data for 32 European countries between 2013 and 2017

In [ ]:
# ============================ Get all data ====================================

Data = pd.read_csv("../Data/13-17_32_conditioned.csv") # National_load_data
Train = pd.read_csv("../Data/13-17_32_Train.csv", index_col=0) # Training data
Test = pd.read_csv("../Data/13-17_32_Test.csv", index_col=0) # Test data

In [ ]:
# ================= Separate load data and time label ==========================

# Get column location
Austria=Data.columns.get_loc('AT_load_actual_entsoe_power_statistics')
Slovakia=Data.columns.get_loc('SK_load_actual_entsoe_power_statistics')
hour_sin=Data.columns.get_loc('hour_sin')
hour_cos=Data.columns.get_loc('hour_cos')

# Get only load data from training data set
Train_load=Train.iloc[:,Austria:(Slovakia+1)]
Train_load.index=range(len(Train_load)) # index rearrange

# Get only load data from test data set
Test_load=Test.iloc[:,Austria:(Slovakia+1)]
Test_load.index=range(len(Test_load)) # index rearrange

# Get only time label from training data set
Train_label=Train.iloc[:,hour_sin:(hour_cos+1)]
Train_label.index=range(len(Train_label)) # index rearrange

# Get only time label from test data set
Test_label=Test.iloc[:,hour_sin:(hour_cos+1)]
Test_label.index=range(len(Test_label)) # index rearrange

In [ ]:
# ============= Data processing for all training and testing data  =============
#============================= Traing data processing ==========================

# To get "normalized" training data
Tr = Train_load
max_Tr = np.max(Tr, axis = 0) # reserve the maximum value of basic training data
min_Tr = np.min(Tr, axis = 0) # reserve the minimum value of basic training data
Tr_nor = (Tr-min_Tr)/(max_Tr-min_Tr) # normalized training data

mean_Tr = np.mean(Tr, axis = 0) # Mean Value

min_Tr_astype = min_Tr.values.astype(np.float32) # Training 
max_Tr_astype = max_Tr.values.astype(np.float32) # Testing 
mean_Tr_astyped = mean_Tr.values.astype(np.float32) # Mean

# To get "normalized" Testing data
Te = Test_load
Te_nor = (Te-min_Tr)/(max_Tr-min_Tr)

# "Date format" conversion for all data
TR_load = Tr_nor.values.astype(np.float32) # Training Data                                                                                                                
TE_load  = Te_nor.values.astype(np.float32) # Testing Data
TR_label = Train_label.values.astype(np.float32) # Training label
TE_label = Test_label.values.astype(np.float32) # Testing label

In [ ]:
# ====== Data processing for specific hour of training and testing data  =======
# ================= Split load data at specific time ===========================

# Define empty list
Index=[None] * 24 # 24 hours
Train_load_at=[None] * 24
Train_load_volume_at=[None] * 24
Test_load_at=[None] * 24
Test_load_volume_at=[None] * 24
# Training load data at specific time
for i in range(len(Index)):
  Index[i] = Train[Train['hour']==i].index.tolist() # Get the index of specific time
  Train_load_at[i]=Train_load.iloc[Index[i]] # Get load data of specific time
  Train_load_at[i].index=range(len(Train_load_at[i])) # Rearrange the load index of data at specific time
  Train_load_volume_at[i]=len(Train_load_at[i]) # Get volume of load data at specific time
# Testing load data at specific time
for i in range(len(Index)):
  Index[i] = Test[Test['hour']==i].index.tolist() # Get the index of specific time
  Test_load_at[i]=Test_load.iloc[Index[i]] # Get load data of specific time
  Test_load_at[i].index=range(len(Test_load_at[i])) # Rearrange the load index of data at specific time
  Test_load_volume_at[i]=len(Test_load_at[i]) # Get volume of load data at specific time

# ================= Split label at specific time ===========================

# Define empty list
Index=[None] * 24 # 24 hours
Train_label_at=[None] * 24
Train_label_volume_at=[None] * 24
Test_label_at=[None] * 24
Test_label_volume_at=[None] * 24
# Training label at specific time
for i in range(len(Index)):
  Index[i] = Train[Train['hour']==i].index.tolist() # Get the index of specific time
  Train_label_at[i]=Train_label.iloc[Index[i]] # Get label of specific time
  Train_label_at[i].index=range(len(Train_label_at[i])) # Rearrange the label index of data at specific time
  Train_label_volume_at[i]=len(Train_label_at[i]) # Get volume of data at specific time
# Testing label at specific time
for i in range(len(Index)):
  Index[i] = Test[Test['hour']==i].index.tolist() # Get the index of specific time
  Test_label_at[i]=Test_label.iloc[Index[i]] # Get label of specific time
  Test_label_at[i].index=range(len(Test_label_at[i])) # Rearrange the label index of data at specific time
  Test_label_volume_at[i]=len(Test_label_at[i]) # Get volume of data at specific time

# To get "normalized" training load data at specific hour
Tr_nor_2 = (Train_load_at[2]-min_Tr)/(max_Tr-min_Tr) 
Tr_nor_10 = (Train_load_at[10]-min_Tr)/(max_Tr-min_Tr)
Tr_nor_21 = (Train_load_at[21]-min_Tr)/(max_Tr-min_Tr)

# Date "format conversion" for hour of training data
# Training load data at 2:00 
TR_load_2 = Tr_nor_2.values.astype(np.float32) 
# Training load data at 10:00
TR_load_10 = Tr_nor_10.values.astype(np.float32) 
# Training load data at 21:00
TR_load_21 = Tr_nor_21.values.astype(np.float32) 

# Training Label at 2:00
TR_label_2 = Train_label_at[2].values.astype(np.float32)
# Training Label at 10:00
TR_label_10 = Train_label_at[10].values.astype(np.float32)
# Training Label at 21:00
TR_label_21 = Train_label_at[21].values.astype(np.float32)

# Preparation (Neuro Network Dessign)

In [ ]:
# ================== Preparation (Neuro network dessign) =======================

# ========================== Initialization fuction ============================

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


# ===================== Data dimension calcuation ==============================

# Dimension of the input
X_dim_all=np.shape(Train_load)
X_dim = X_dim_all[1]
# Dimension of the time label
y_dim_all =np.shape(TR_label) 
y_dim = y_dim_all[1]
# Dimension of the hidden layer
h_dim_1 = 24
# Dimension of the hidden layer
h_dim_2 = 16
# Dimension of the latent space
z_dim = 8

# ============================= Print the dimension ============================

print ('The dimension of the input data is: ', X_dim)
print ('The dimension of the hidden layer 1 is: ', h_dim_1)
print ('The dimension of the hidden layer 2 is: ', h_dim_2)
print ('The dimension of the latent space data is: ', z_dim)

# =============================== Q(z|X,c) =====================================

# Data
X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, y_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

# Layer definitions
Q_W1 = tf.Variable(xavier_init([X_dim+y_dim, h_dim_1]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim_1]))

Q_W2 = tf.Variable(xavier_init([h_dim_1, h_dim_2]))
Q_b2 = tf.Variable(tf.zeros(shape=[h_dim_2]))

Q_W3_mu = tf.Variable(xavier_init([h_dim_2, z_dim]))
Q_b3_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W3_sigma = tf.Variable(xavier_init([h_dim_2, z_dim]))
Q_b3_sigma = tf.Variable(tf.zeros(shape=[z_dim]))

# Network
def Q(X, c): 
    inputs = tf.concat(axis=1, values=[X, c]) 
    h1 = tf.nn.relu((tf.matmul(inputs, Q_W1) + Q_b1))
    h2 = tf.nn.relu((tf.matmul(h1, Q_W2) + Q_b2))
    z_mu = tf.matmul(h2, Q_W3_mu) + Q_b3_mu
    z_logvar = tf.matmul(h2, Q_W3_sigma) + Q_b3_sigma
    return z_mu, z_logvar

# ================Sampling a z from N~(z_mu|tf.exp(z_logvar))===================

def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps

# =============================== P(X|z,c) =======================================

# Layer definitions
P_W1 = tf.Variable(xavier_init([z_dim + y_dim , h_dim_2]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim_2]))

P_W2 = tf.Variable(xavier_init([h_dim_2 , h_dim_1]))  
P_b2 = tf.Variable(tf.zeros(shape=[h_dim_1]))

P_W3_mu = tf.Variable(xavier_init([h_dim_1, X_dim]))
P_b3_mu = tf.Variable(tf.zeros(shape=[X_dim]))

P_W3_logvar = tf.Variable(xavier_init([h_dim_1, X_dim]))
P_b3_logvar = tf.Variable(tf.zeros(shape=[X_dim]))

# Define the manually-set σ as a hyperparameter
σ_manually_set=0.1 # manually-set σ

# Network
def P(z,c):
    inputs = tf.concat(axis=1, values=[z, c]) 
    h1 = tf.nn.relu((tf.matmul(inputs, P_W1) + P_b1))
    h2 = tf.nn.relu((tf.matmul(h1, P_W2) + P_b2))
    Gen_mu = tf.matmul(h2, P_W3_mu) + P_b3_mu # Gen_mu : μ_x(z) 
    eps_p = tf.random_normal(shape=tf.shape(Gen_mu)) # epsilon
    Gen_σ=tf.ones(shape=tf.shape(Gen_mu))*σ_manually_set # Gen_σ:σ (manually-set σ of all dimensions)
    Gen_noisy = Gen_mu +eps_p*Gen_σ # Gen_noisy = μ_x(z) + eps_p* σ 
    return Gen_mu, Gen_noisy    


# Training (CVAE; Manually set σ')

In [ ]:
# ================================ Training ====================================

# Training condition: lr (learning rate)=0.0001; batch_size (mb_size) =64;
#                     iteration=200,000; σ' Manually set

#Traning setting 
mb_size = 64 # Batch size
lr = 0.0001 #learning rate
β = 1 # weight ratio of kl_loss and recon_loss 
iteration = 200000 # define the iteration steps

# Training functions
z_mu, z_logvar = Q(X, c) # z follows N~(z_μ,z_log(σ^2)) 
z_sample = sample_z(z_mu, z_logvar) # Sampling z_sample from N~(z_μ, tf.exp(z_log(σ^2)))
Gen_mu_train, Gen_noisy_train = P(z_sample, c) # generate (hat) x_μ, x_log(σ^2), x given z_sample,c

# Generation functions
Gen_mu_sample, Gen_noisy_sample = P(z, c) # Sampling (tilde) x_μ, x_log(σ^2), x given z follows N~(0|1),c 

# Training loss when σ' is manually set
recon_loss = 0.5 *tf.reduce_sum((Gen_mu_train - X)**2/(σ_manually_set**2)+tf.math.log(σ_manually_set**2), 1) # recon_error

kl_loss  = β*0.5 * tf.reduce_sum(tf.exp(z_logvar) - 1. - z_logvar + z_mu**2, 1) # KL_error

cvae_loss = tf.reduce_mean(recon_loss + kl_loss)+X_dim/2*tf.log(2*(math.pi)) # CVAE loss

# Monitor what the exact number 
recon_loss_mean = tf.reduce_mean(recon_loss)+X_dim/2*tf.log(2*(math.pi))

kl_loss_mean = tf.reduce_mean(kl_loss)

# Solver
solver = tf.train.AdamOptimizer(lr).minimize(cvae_loss)

# Seesion
sess = tf.Session() # Session definition
sess.run(tf.global_variables_initializer())

for it in range(iteration): # Training process
    batch_index = np.random.choice(TR_load.shape[0], size=mb_size, replace=False)
    _, loss, KL_error, Recon_error = sess.run([solver, cvae_loss, kl_loss_mean, recon_loss_mean], 
                                              feed_dict={X: TR_load[batch_index], c:TR_label[batch_index]})                                                                                                                    
    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        print('KL_error: {}'. format(KL_error))
        print('Recon_error: {}'. format(Recon_error))
        print()
print("Training Finished!") 

#Save the reslut    
saver = tf.train.Saver() 
saver.save(sess, './filename.chkp') 

# Session Recover
sess = tf.Session()
saver = tf.train.Saver()                                                     
saver.restore(sess, 'filename.chkp')

# Load data Generation

In [ ]:
# ==========================Generate load data==================================

# =======================Function to recover data===============================

def Gen_recover_function(Gen_to_be_recovered): 
    Gen_recovered=Gen_to_be_recovered*(max_Tr_astype-min_Tr_astype)+min_Tr_astype
    return Gen_recovered

# =========================Geneter load data from 0-23 o'clock ==================
Gen_mu_at=[None]* 24
Gen_mu_recovered_at=[None] * 24

Gen_noisy_at=[None]* 24
Gen_noisy_recovered_at=[None] * 24

for Time in range(len(Gen_mu_at)):   
    # Get labels
    GEN_label = np.zeros(shape=[Train_load_volume_at[Time], y_dim])
    GEN_label[:,0]=np.sin(Time/12*math.pi).round(2)
    GEN_label[:,1]=np.cos(Time/12*math.pi).round(2)
    # Generate data
    Gen_mu_new, Gen_noisy_new = sess.run([Gen_mu_sample, Gen_noisy_sample], 
                                             feed_dict={z: np.random.randn(Train_load_volume_at[Time],z_dim), c: GEN_label})
        
    # Generate data mu (rescaled）
    Gen_mu_at[Time] = Gen_mu_new # Gen_mu_new
    Gen_mu_recovered_at[Time] =  Gen_recover_function(Gen_mu_new) # Gen_mu_new (rescaled)
    
    # Generate data noisy (rescaled）
    Gen_noisy_at[Time] = Gen_noisy_new # Gen_noisy_new 
    Gen_noisy_recovered_at[Time] = Gen_recover_function(Gen_noisy_new) # Gen_noisy_new (rescaled)

    if Time == 0:
      Gen_mu_all = Gen_mu_at[Time]    # All generated data mu
      Gen_mu_recovered_all = Gen_mu_recovered_at[Time]   # All generated data mu (rescaled)
      Gen_noisy_all = Gen_noisy_at[Time] # All generated data noisy
      Gen_noisy_recovered_all = Gen_noisy_recovered_at[Time]   # All generated data noisy (rescaled)
    else:
      Gen_mu_all = np.concatenate((Gen_mu_all, Gen_mu_at[Time]),axis=0)    
      Gen_mu_recovered_all = np.concatenate((Gen_mu_recovered_all, Gen_mu_recovered_at[Time]),axis=0)  
      Gen_noisy_all = np.concatenate((Gen_noisy_all, Gen_noisy_at[Time]),axis=0)    
      Gen_noisy_recovered_all = np.concatenate((Gen_noisy_recovered_all, Gen_noisy_recovered_at[Time]),axis=0)  

# Explanation for the file names
- filename with "σ'_auto": output noise parameter is co-optimised during training (Auto σ')
- filename with "σ'_"+ a "number": output noise parameter is  fixed (Manually set σ')

- filename with "mu": generated data without noise (noise free)
- filename with "noisy": generated data with noise (noisy)

- filename with "Time_all": generated data for 24 hours
- filename with "Time_"+ a "number": generated data for a specific hour of the day

- filename with "nor": generated data that are not rescaled to the original scale of historical data
- filename without "nor": generated data that are rescaled to the original scale of historical data

- filename with a β number: data generated with that specific β number
- filename without a β number: the number of β is 1 by default

- filename with "VAE" specified: it is a VAE model
- filename without "VAE" specified: it is a CVAE model by default

- filename with "35" in the end: a model for 35-dimensional input
- filename without "35" in the end: a model for 32-dimensional input by default

In [ ]:
# Save data
pd.DataFrame(Gen_mu_recovered_at[2]).to_csv("σ'_{:.2f}_Time_2_mu.csv".format(σ_manually_set))
pd.DataFrame(Gen_mu_recovered_at[10]).to_csv("σ'_{:.2f}_Time_10_mu.csv".format(σ_manually_set))
pd.DataFrame(Gen_mu_recovered_at[21]).to_csv("σ'_{:.2f}_Time_21_mu.csv".format(σ_manually_set))
pd.DataFrame(Gen_noisy_recovered_at[2]).to_csv("σ'_{:.2f}_Time_2_noisy.csv".format(σ_manually_set))
pd.DataFrame(Gen_noisy_recovered_at[10]).to_csv("σ'_{:.2f}_Time_10_noisy.csv".format(σ_manually_set))
pd.DataFrame(Gen_noisy_recovered_at[21]).to_csv("σ'_{:.2f}_Time_21_noisy.csv".format(σ_manually_set))
pd.DataFrame(Gen_mu_recovered_all).to_csv("σ'_{:.2f}_Time_all_mu.csv".format(σ_manually_set))
pd.DataFrame(Gen_noisy_recovered_all).to_csv("σ'_{:.2f}_Time_all_noisy.csv".format(σ_manually_set))
pd.DataFrame(Gen_mu_all).to_csv("σ'_{:.2f}_Time_all_mu_nor.csv".format(σ_manually_set))
pd.DataFrame(Gen_noisy_all).to_csv("σ'_{:.2f}_Time_all_noisy_nor.csv".format(σ_manually_set))